In [92]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [112]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH, sub_sample=True)

## Do your thing crazy machine learning thing here :) ...

In [113]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from logistic_regression import *
from reg_logistic_regression import *

# tX_norm = normalize(tX)
# pca = PCA(n_components=30)
# pca.fit(tX_norm)
# print(np.sum(pca.explained_variance_ratio_))
# tX_proj = pca.transform(tX)

clf = LogisticRegression(random_state=0).fit(tX, y)
print("Accuracy from scikit learn: "+str(round(100*clf.score(tX, y),5))+' %\n')

#Using our code
tX = correction_missing_values(tX)
tX, mean_x, std_x = standardize(tX)
tX = np.c_[np.ones((y.shape[0], 1)), tX]

gamma = 0.1
lamda_ = 10
n_iter = 1000


w = np.random.rand(tX.shape[1])
for i in range(n_iter):
    w, loss = learning_by_gradient_descent(y, tX, w, gamma)

print("Accuracy from our code:\n")
print("Logistic regression: "+str(round(100*(1-np.linalg.norm(predict_labels(w, tX)-y,1)/2/len(y)),5))+" %\n")

w_reg = w
for i in range(n_iter):
    w_reg, loss = learning_by_penalized_gradient(y, tX, w_reg, gamma, lamda_)

print("After regularization: "+str(round(100*(1-np.linalg.norm(predict_labels(w_reg, tX)-y,1)/2/len(y)),5))+" %\n")

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Accuracy from scikit learn: 74.6 %



/home/ML_course/projects/project1/scripts/logistic_regression.py:12: RuntimeWarning: divide by zero encountered in log
  return np.squeeze(-(np.dot(y.transpose(), np.log(sigmoid(np.dot(tx, w)))) + np.dot((1 - y).transpose(), np.log(1 - sigmoid(np.dot(tx, w))))))
/home/ML_course/projects/project1/scripts/logistic_regression.py:8: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))


Accuracy from our code:

Logistic regression: 85.86 %



/home/ML_course/projects/project1/scripts/reg_logistic_regression.py:8: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1 + np.exp(-t))
/home/ML_course/projects/project1/scripts/reg_logistic_regression.py:12: RuntimeWarning: divide by zero encountered in log
  return np.squeeze(-(np.dot(y.transpose(), np.log(sigmoid(np.dot(tx, w)))) + np.dot((1 - y).transpose(), np.log(1 - sigmoid(np.dot(tx, w))))))


After regularization: 82.91 %



## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)